In [1]:
import numpy as np
from skimage.transform import rotate , resize
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
path = "/home/parth/ml/handson-ml/datasets/inception/inception_v3.ckpt"

In [4]:
import os

names = os.listdir("/home/parth/ml/Tensorflow_learning/Hands on Ml/flower_photos")
names.remove(names[2])
class_to_value = { name : value for value , name in enumerate(names)}
class_to_value         

{'daisy': 1, 'dandelion': 0, 'roses': 2, 'sunflowers': 4, 'tulips': 3}

In [ ]:
def read_data():
    data_ = dict()
    for name in names :
        path = "/home/parth/ml/Tensorflow_learning/Hands on Ml/flower_photos" + "/" + name 
        filenames = os.listdir(path)
        temp_list = list()
        for file in filenames :
            file = path + "/" + file
            temp_list.append(plt.imread(file , format="jpg"))
        data_[name] = temp_list
    return data_

In [ ]:
def get_training_data():
    data_ = read_data()
    X = list()
    Y = list()
    for label in data_.keys():
        print("Doing Label : {}".format(label))
        x = data_[label]
        for eachx in x:
            eachx.astype(np.uint16 , copy=False)
            eachx = resize(eachx , [299 , 299 , 3])
            X.append(eachx)
            Y.append(class_to_value[label])
            print(a)
    return X , Y

In [ ]:
Xtrain , ytrain = get_training_data()

In [ ]:
Xtrain = np.array(Xtrain)

In [ ]:
ytrain = np.array(ytrain)

In [ ]:
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, 299, 299, 3], name="X")
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(
        X, num_classes=1001, is_training=False)
predictions = end_points["Predictions"]
file_writer = tf.summary.FileWriter(logdir="/tmp/")
saver = tf.train.import_meta_graph(path)

In [ ]:
import re
import os

CLASS_NAME_REGEX = re.compile(r"^n\d+\s+(.*)\s*$", re.M | re.U)

def load_class_names():
    path = os.path.join("/home/parth/ml/handson-ml/datasets", "inception", "imagenet_class_names.txt")
    with open(path, encoding="utf-8") as f:
        content = f.read()
        return CLASS_NAME_REGEX.findall(content)
    
class_names = ["background"] + load_class_names()

In [ ]:
with tf.Session() as sess:
    saver.restore(sess , path)
    file_writer.add_graph(tf.get_default_graph())

In [ ]:
outputs = tf.get_default_graph().get_operation_by_name("InceptionV3/InceptionV3/Mixed_7b/concat")
Layer = outputs.outputs[0]

In [ ]:
layer = tf.stop_gradient(Layer)
layer = tf.layers.conv2d(layer , activation=tf.nn.relu , kernel_size=1 , filters=1024  , strides=1 , padding="SAME")
layer = tf.layers.conv2d(layer , activation=tf.nn.relu , kernel_size=3 , filters=1024 , strides=2 , padding="SAME")
layer = tf.nn.local_response_normalization(layer , depth_radius=2)
layer = tf.layers.conv2d(layer , activation=tf.nn.relu , kernel_size=1 , filters=256  , strides=1 , padding="SAME")
layer = tf.layers.conv2d(layer , activation=tf.nn.relu , kernel_size=3 , filters=512  , strides=1,  padding="SAME")
layer = tf.layers.average_pooling2d(layer , pool_size=2 , strides=2 , padding="VALID")

In [ ]:
flatten = tf.layers.flatten(layer)
dense = tf.layers.dense(flatten , units=100 , 
                        activation=tf.nn.elu , kernel_initializer=tf.variance_scaling_initializer())
logits_new = tf.layers.dense(dense , units=5  , kernel_initializer=tf.variance_scaling_initializer())

y = tf.placeholder(tf.int32, shape=(None) , name="y")

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits_new))

optimzer = tf.train.AdamOptimizer()
training_op = optimzer.minimize(loss)

acc = tf.reduce_mean(tf.cast(tf.math.in_top_k(logits , tf.cast(y , tf.float32) , 1) , tf.float32))
init = tf.global_variables_initializer()

In [ ]:
before = 0
with tf.Session() as sess :
            sess.run(init)
            for epoch in range(n_epoch):
                perms = np.random.permutation(num_examples)
                Xbatches = np.array_split(Xtrain[perms] , num_examples // batch_size)
                ybatches = np.array_split(ytrain[perms] , num_examples // batch_size )
                for Xbatch , ybatch in zip(Xbatches , ybatches):
                    feed_dict = {X:Xbatch , y:ybatch}
                    accu , _ = sess.run([acc , training_op] , feed_dict=feed_dict)
                    if accu > before:
                        before = accu
                        saver.save(sess , "/tmp/model.ckpt")
                print(accu)